In [18]:
import pandas
from create_train_test_val_maps import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

s_to_date = lambda x: pandas.to_datetime(x, format='%Y/%m/%d %H:%M:%S')
ok_snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_ok_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

In [13]:
ata6_train_repairs = pandas.read_csv('ATA6_train.csv')
ata6_val_repairs = pandas.read_csv('ATA6_val.csv')
ata6_test_repairs = pandas.read_csv('ATA6_test.csv')

category_var = ['Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','DTCID']
for cat in category_var:
    snapshots[cat] = snapshots[cat].astype('category').cat.codes
    ok_snapshots[cat] = ok_snapshots[cat].astype('category').cat.codes

try:
    snapshots = snapshots.drop(['DTC'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')
    
try:
    ok_snapshots = ok_snapshots.drop(['DTC'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')

train_veh_ids = ata6_train_repairs['Chassis Reference Number'].unique().astype(int)
val_veh_ids = ata6_val_repairs['Chassis Reference Number'].unique().astype(int)
test_veh_ids = ata6_test_repairs['Chassis Reference Number'].unique().astype(int)

all_veh_ids = ok_snapshots['Veh Ref ID'].unique().astype(int)
train_ok_ids = all_veh_ids[0:100]
val_ok_ids = all_veh_ids[100:120]
test_ok_ids = all_veh_ids[120:144]

ids_tuple = (train_veh_ids, val_veh_ids, test_veh_ids)
ids_ok_tuple = (train_ok_ids, val_ok_ids, test_ok_ids)
repairs_tuple = (ata6_train_repairs, ata6_val_repairs, ata6_test_repairs)

In [ ]:
# run this first in your terminal to clear out previous maps
# > rm /home/cs231n/data/train*; rm /home/cs231n/data/val*; rm /home/cs231n/data/test*;

In [76]:
ok_size_dist = (24, 30, 17) # do not change this

hyper_tuples = [(1,500), (5,20), (5,50), (2,250), (5,100), (5,250), (10, 10), (10, 100)]

for num_windows, window_size in hyper_tuples:
    create_maps(num_windows, window_size, snapshots, ok_snapshots, ids_tuple, ids_ok_tuple, repairs_tuple, ok_size_dist)

In [77]:
train_combined, val_combined, test_combined = load_maps(10, 10)